In [1]:
using StreamOps

In [6]:
function run_test()

    pipe = @streamops EWMean{Float64}(; alpha=0.1) Diff{Float64}(1) Func(x -> x + 1)
    # pipe = OpFunc(x -> x + 1; next=OpEWMean{Float64}(0.1)) 

    pipe(1.0)

    @time for i in 1.0:1_000_000.0
        pipe(i)
    end

    pipe(1.0)
end

run_test()

  0.032935 seconds


-99998.0

In [7]:
@macroexpand @streamops Func(x -> x + 1) EWMean{Float64}(; alpha=0.1)

quote
    #= /mnt/data/repos/StreamOps.jl/src/macros.jl:291 =#
    local var"#126###state#255" = Func((x->begin
                        #= /mnt/data/repos/StreamOps.jl/test_ops.ipynb:1 =#
                        x + 1
                    end))
    local var"#127###state#257" = EWMean{Float64}(; alpha = 0.1)
    #= /mnt/data/repos/StreamOps.jl/src/macros.jl:292 =#
    function (var"#130#x"::var"#131#T",) where var"#131#T"
        #= /mnt/data/repos/StreamOps.jl/src/macros.jl:292 =#
        #= /mnt/data/repos/StreamOps.jl/src/macros.jl:293 =#
        begin
            local var"#128###call#256" = var"#126###state#255"(var"#130#x")
            local var"#129###call#258" = var"#127###state#257"(var"#128###call#256")
        end
    end
end

In [ ]:
using BenchmarkTools

function test_manual(count)
	last::Ref{Float64} = Ref(Float64(NaN))
	pipeline = function(x)
		x = abs(x)
		x = x * x
		tmp = last[]
		last[] = x
		tmp
	end
	current::Float64 = Float64(NaN)
	for i in 1:count
		current = pipeline(Float64(i))
	end
	current
end

function test_pipeline(count)
	pipeline = @pipeline OpAbs OpPrev{Float64} OpReturn
	current::Float64 = Float64(NaN)
	for i in 1:count
		current = pipeline(Float64(i))
	end
	current
end
display(@benchmark test_manual(1e6) samples = 30 evals = 3)
display(@benchmark test_pipeline(1e6) samples = 30 evals = 3)

In [ ]:
# using ProfileView
# ProfileView.@profview map(x -> test(1e7), 1:10)

# using Profile, ProfileSVG
# @Profile.profile test_pipeline(1e8)
# ProfileSVG.view(maxdepth=300, maxframes=3000)